<h1 align=center><font size = 5>Deep Belief Network </font></h1>

Os __DBNs__ podem ser divididos em duas partes principais. O primeiro deles são múltiplas camadas de RBMs (Restricted Boltzmann Machines) para pré-treinar nossa rede. O segundo é uma rede backpropagation, que irá refinar ainda mais os resultados da pilha RBM.

<img src="dbn.jpg" alt="DBN Model"/>

In [1]:
#math para calculos
import math
#Tensorflow para os modelos de machine learn
import tensorflow as tf
#Numpy para calculos matematicos
import numpy as np

Para implementar a rede DBN , implementarei uma classe para as Máquinas Restritas da Boltzmann (RBM). A classe abaixo implementa uma maneira de criar e usar RBMs.

In [2]:
# definindo a classe maquina de boltzman restrita
class RBM(object):

    def __init__(self, input_size, output_size):
        # definindo hiperparametros
        self._input_size = input_size
        self._output_size = output_size
        self.epochs = 5  # numero de interacoes do treino
        self.learning_rate = 1.0  # passo usado no gradiente descendente
        self.batchsize = 100  # quantia de dados a serem usados no treino por sub-interacao

        # Iniciando os pesos e bias como matrizes de zeros
        self.w = np.zeros([input_size, output_size], np.float32)  # inicializando os pesos com zero
        self.hb = np.zeros([output_size], np.float32)  # inicializando as bias ocultas com zero
        self.vb = np.zeros([input_size], np.float32)  # inicializando as bias visiveis com zero

    # ajustando o resultado da camada visivel(aplicada a esta os pesos) e as bias, em uma curva sigmoid
    def prob_h_given_v(self, visible, w, hb):
        # Sigmoid
        return tf.nn.sigmoid(tf.matmul(visible, w) + hb)

    # ajustando o resultado da camada oculta(aplicada a esta os pesos) e as bias, em uma curva sigmoid
    def prob_v_given_h(self, hidden, w, vb):
        return tf.nn.sigmoid(tf.matmul(hidden, tf.transpose(w)) + vb)

    # gerando a probabilidade por amostra
    def sample_prob(self, probs):
        return tf.nn.relu(tf.sign(probs - tf.random_uniform(tf.shape(probs))))#Adicionando nao linearidade, transformacao linear onde tod mudanca nos inputs e proporcional as dos outputs

    # metodo de treino para o modelo
    def train(self, X):
        # criando espaco exclusivo para os parametros
        _w = tf.placeholder("float", [self._input_size, self._output_size])
        _hb = tf.placeholder("float", [self._output_size])
        _vb = tf.placeholder("float", [self._input_size])

        anterior_w = np.zeros([self._input_size, self._output_size],
                         np.float32)  # Creates and initializes the weights with 0
        anterior_hb = np.zeros([self._output_size], np.float32)  # Cria e inicializa as bias ocultas com 0
        anterior_vb = np.zeros([self._input_size], np.float32)  # Cria e inicializa as bias visíveis com 0

        atual_w = np.zeros([self._input_size, self._output_size], np.float32)
        atual_hb = np.zeros([self._output_size], np.float32)
        atual_vb = np.zeros([self._input_size], np.float32)
        v0 = tf.placeholder("float", [None, self._input_size])

        # inicializando com as probabilidades por amostra
        h0 = self.sample_prob(self.prob_h_given_v(v0, _w, _hb))
        v1 = self.sample_prob(self.prob_v_given_h(h0, _w, _vb))
        h1 = self.prob_h_given_v(v1, _w, _hb)

        # criando gradientes
        positive_grad = tf.matmul(tf.transpose(v0), h0)
        negative_grad = tf.matmul(tf.transpose(v1), h1)

        # atualizando a taxa(pontuacao) de aprendizado por camada
        update_w = _w + self.learning_rate * (positive_grad - negative_grad) / tf.to_float(tf.shape(v0)[0])
        update_vb = _vb + self.learning_rate * tf.reduce_mean(v0 - v1, 0)
        update_hb = _hb + self.learning_rate * tf.reduce_mean(h0 - h1, 0)

        # calcular erro
        err = tf.reduce_mean(tf.square(v0 - v1))

        # loop do treino
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            # para cada epoch
            for epoch in range(self.epochs):
                # para cada passo step/batch
                for start, end in zip(range(0, len(X), self.batchsize), range(self.batchsize, len(X), self.batchsize)):
                    batch = X[start:end]
                    # atualizando pontuacao
                    atual_w = sess.run(update_w, feed_dict={v0: batch, _w: anterior_w, _hb: anterior_hb, _vb: anterior_vb})
                    atual_hb = sess.run(update_hb, feed_dict={v0: batch, _w: anterior_w, _hb: anterior_hb, _vb: anterior_vb})
                    atual_vb = sess.run(update_vb, feed_dict={v0: batch, _w: anterior_w, _hb: anterior_hb, _vb: anterior_vb})
                    anterior_w = atual_w
                    anterior_hb = atual_hb
                    anterior_vb = atual_vb
                error = sess.run(err, feed_dict={v0: X, _w: atual_w, _vb: atual_vb, _hb: atual_hb})
                print ('Epoch: %d' % epoch, 'reconstruction error: %f' % error)
            self.w = anterior_w
            self.hb = anterior_hb
            self.vb = anterior_vb

    # saida necessaria para a DBN
    def rbm_outpt(self, X):
        input_X = tf.constant(X)
        _w = tf.constant(self.w)
        _hb = tf.constant(self.hb)
        out = tf.nn.sigmoid(tf.matmul(input_X, _w) + _hb)
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            return sess.run(out)

In [3]:
#Carregando os dados
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data", one_hot=True)
trX, trY, teX, teY = mnist.train.images, mnist.train.labels, mnist.test.images, mnist.test.labels

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [4]:
#Criando a DBN
RBM_hidden_sizes = [500, 200 , 50 ] #cria 3 RBMs, uma com 500 unidades ocultas, segunda com 200 e a ultima com 50 
#Inputs serao os dados de treino
inpX = trX

#Criando lista para guardar as RBMs
rbm_list = []

#Tamanho das entradas é o número de entradas no conjunto de treinamento
input_size = inpX.shape[1]

#Para cada RBM sera inicializada atribuindo o tamnaho do conj treino e o numero de unidades ocultas,para segunda RBM o tamanho do conj treino recebera o valor do numero de unidades ocultas da camada anterior, assim por diante
for i, size in enumerate(RBM_hidden_sizes):
    print ('RBM: ',i,' ',input_size,'->', size)
    rbm_list.append(RBM(input_size, size))
    input_size = size #Serve para forcar compressao dos dados

RBM:  0   784 -> 500
RBM:  1   500 -> 200
RBM:  2   200 -> 50


In [5]:
#para cada RBM iniciar o treinar, uasando sempre como entrada de treino da proxima RBM o output da anterior
for rbm in rbm_list:
    print ('New RBM:')
    #treinando nova RBM
    rbm.train(inpX) 
    #Retornando output
    inpX = rbm.rbm_outpt(inpX)

New RBM:
Instructions for updating:
Use tf.cast instead.
Epoch: 0 reconstruction error: 0.060681
Epoch: 1 reconstruction error: 0.052861
Epoch: 2 reconstruction error: 0.048948
Epoch: 3 reconstruction error: 0.047465
Epoch: 4 reconstruction error: 0.045820
New RBM:
Epoch: 0 reconstruction error: 0.035045
Epoch: 1 reconstruction error: 0.030750
Epoch: 2 reconstruction error: 0.028657
Epoch: 3 reconstruction error: 0.027566
Epoch: 4 reconstruction error: 0.027030
New RBM:
Epoch: 0 reconstruction error: 0.056838
Epoch: 1 reconstruction error: 0.052835
Epoch: 2 reconstruction error: 0.051468
Epoch: 3 reconstruction error: 0.050643
Epoch: 4 reconstruction error: 0.050701


In [6]:
#Com a DBN treinada cria-se a Rede neural que fara uso das RBMs da DBN
class NN(object):
    
    def __init__(self, sizes, X, Y):
        #Inicializando hyperparametros
        self._sizes = sizes
        self._X = X
        self._Y = Y
        self.w_list = []
        self.b_list = []
        self._learning_rate =  1.0
        self._momentum = 0.0
        self._epoches = 10
        self._batchsize = 100
        input_size = X.shape[1]
        
        #Loop de iinicializacao
        for size in self._sizes + [Y.shape[1]]:
            #Definindo limite superior da distribuicao uniforme
            max_range = 4 * math.sqrt(6. / (input_size + size))
            
            #Inicializando os pesos como uma distribuicao uniforme aleatoria
            self.w_list.append(
                np.random.uniform( -max_range, max_range, [input_size, size]).astype(np.float32))
            
            #Inicializando as bias como zeros
            self.b_list.append(np.zeros([size], np.float32))
            input_size = size
      
    #lendo dados da RBm
    def load_from_rbms(self, dbn_sizes,rbm_list):
        #Checando os tamanhos esperados
        assert len(dbn_sizes) == len(self._sizes)
        
        for i in range(len(self._sizes)):
            #Checando para cada RBM se o tamanho esta correto
            assert dbn_sizes[i] == self._sizes[i]
        
        #Carregando bias e pesos
        for i in range(len(self._sizes)):
            self.w_list[i] = rbm_list[i].w
            self.b_list[i] = rbm_list[i].hb

    #treino 
    def train(self):
        print()
        #Cria espaçcs reservados para entrada, pesos, bias, saidas
        _a = [None] * (len(self._sizes) + 2)
        _w = [None] * (len(self._sizes) + 1)
        _b = [None] * (len(self._sizes) + 1)
        _a[0] = tf.placeholder("float", [None, self._X.shape[1]])
        y = tf.placeholder("float", [None, self._Y.shape[1]])
        
        #Definindo variaveis e funcoes de ativacoes
        for i in range(len(self._sizes) + 1):
            _w[i] = tf.Variable(self.w_list[i])
            _b[i] = tf.Variable(self.b_list[i])
        for i in range(1, len(self._sizes) + 2):
            _a[i] = tf.nn.sigmoid(tf.matmul(_a[i - 1], _w[i - 1]) + _b[i - 1])
        
        #Funcao custo
        cost = tf.reduce_mean(tf.square(_a[-1] - y))
        
        #Definindo operacao de treino (Momentum Optimizer minimizing the Cost function)
        train_op = tf.train.MomentumOptimizer(self._learning_rate, self._momentum).minimize(cost)
        
        #Operacao de predicao
        predict_op = tf.argmax(_a[-1], 1)
        
        #Loop de treino
        with tf.Session() as sess:
            #Initicializando variaveis
            sess.run(tf.global_variables_initializer())
            
            #Para cada epoca
            for i in range(self._epoches):
                print("is here")
                #Para cada passo
                for start, end in zip(range(0, len(self._X), self._batchsize), range(self._batchsize, len(self._X), self._batchsize)):
                    
                    #Executando operacao de treino em cada input
                    sess.run(train_op, feed_dict={ _a[0]: self._X[start:end], y: self._Y[start:end]})
                
                for j in range(len(self._sizes) + 1):
                    #recuperando pesos e bias
                    self.w_list[j] = sess.run(_w[j])
                    self.b_list[j] = sess.run(_b[j])
                
                print ("Precisao por epoca " + str(i) + ": " + str(np.mean(np.argmax(self._Y, axis=1) == sess.run(predict_op, feed_dict={_a[0]: self._X, y: self._Y}))))

In [7]:
Rede_Neural = NN(RBM_hidden_sizes, trX, trY)
Rede_Neural.load_from_rbms(RBM_hidden_sizes,rbm_list)
Rede_Neural.train()


Instructions for updating:
Colocations handled automatically by placer.
is here
Precisao por epoca 0: 0.49334545454545453
is here
Precisao por epoca 1: 0.6391272727272728
is here
Precisao por epoca 2: 0.7171636363636363
is here
Precisao por epoca 3: 0.7567272727272727
is here
Precisao por epoca 4: 0.7858909090909091
is here
Precisao por epoca 5: 0.8438727272727272
is here
Precisao por epoca 6: 0.8820727272727272
is here
Precisao por epoca 7: 0.8942181818181818
is here
Precisao por epoca 8: 0.9032181818181818
is here
Precisao por epoca 9: 0.9095454545454545
